In [1]:
!pip install cohere

INFO: pip is looking at multiple versions of pydantic to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.0/147.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.4/222.4 kB 2.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 1.9 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 1.1 MB/s eta 0:00:0000:0100:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.4/434.4 kB 4.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 6.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 4.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 5.8 MB/s eta 0:00:00
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.14.6
    Uninstalling pydantic_core-2.14.6:
      Successfully uninstalled pydant

In [3]:
import cohere

co = cohere.Client('DbUauyI3zFWlrXUhOrZpB2yFZszSUs8wlmbZqGfl')

In [103]:
def generate_flashcards(text):
    summary_generated = co.generate(
        model='command-xlarge-nightly',
        prompt=f"Summarize the following text:\n\n{text}",
        max_tokens=150,
        num_generations=1,
        temperature=0.5
    )
    summary = summary_generated.generations[0].text.strip()
    questions_response = co.generate(
        model='command-xlarge-nightly',
        prompt=f"Create a list of questions from the following summary:\n\n{summary}",
        max_tokens=100,
        num_generations=5, 
        temperature=0.7
    )
    questions = [gen.text.strip() for gen in questions_response.generations]
    reranked_questions = co.rerank(
        query=f"Select the most relevant questions from the following list:",
        documents=questions, 
        top_n=3,  
        return_documents=True  
    )
    # for result in reranked_questions.results:
    #     print(f"Document: {result.document.text}")
    #     print(f"Relevance Score: {result.relevance_score}\n")
    best_result = max(reranked_questions.results, key=lambda x: x.relevance_score)
    text_with_questions = best_result.document.text
    selected_questions = [line.strip() for line in text_with_questions.splitlines() if line.startswith('-')]
    # for question in questions:
    #     print(question)
    flashcards = []
    for question in selected_questions:
        answer_response = co.generate(
            model='command-xlarge-nightly',
            prompt=f"Provide a concise answer to the following question based on the text: {text}\n\nQuestion: {question}",
            max_tokens=100,
            num_generations=1,  
            temperature=0.7
        )
        answer = answer_response.generations[0].text.strip()
        flashcards.append({
            "question": question,
            "answer": answer
        })

    return flashcards
    

In [105]:
text = 'Battle of Waterloo, (June 18, 1815), Napoleon’s final defeat, ending 23 years of recurrent warfare between France and the other powers of Europe. It was fought during the Hundred Days of Napoleon’s restoration, 3 miles (5 km) south of Waterloo village (which is 9 miles [14.5 km] south of Brussels), between Napoleon’s 72,000 troops and the combined forces of the duke of Wellington’s allied army of 68,000 (with British, Dutch, Belgian, and German units) and about 45,000 Prussians, the main force of Gebhard Leberecht von Blücher’s command.'
sum = generate_flashcards(text)
sum

[{'question': '- When and where did the Battle of Waterloo take place?',
  'answer': 'The Battle of Waterloo took place on June 18, 1815, about 3 miles south of Waterloo village, which is 9 miles south of Brussels.'},
 {'question': '- Who were the key military leaders involved in the Battle of Waterloo?',
  'answer': 'The key military leaders involved in the Battle of Waterloo were Napoleon, the duke of Wellington, and Gebhard Leberecht von Blücher.'},
 {'question': '- How many troops did Napoleon command at Waterloo?',
  'answer': 'Napoleon commanded 72,000 troops at Waterloo.'},
 {'question': '- Which nations were part of the coalition army led by the Duke of Wellington?',
  'answer': 'The coalition army led by the Duke of Wellington included British, Dutch, Belgian, and German units.'},
 {'question': '- What was the approximate size of the Prussian force at Waterloo?',
  'answer': 'The Prussian force at Waterloo was approximately 45,000 troops.'},
 {'question': "- What was the outco

In [107]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 10.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 11.6 MB/s eta 0:00:00a 0:00:01


In [117]:
import pdfplumber

def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text()
    return text

pdf_path = "/Users/arpitsrivastav/Desktop/hack-the-north-2024/Stat240Fall2024W1.pdf"
pdf_text = extract_text_from_pdf(pdf_path)
print(pdf_text)

STAT 240 Week 1 Summary
1 Probability (vs Statistics)
Probability starts with a model, and it studies properties of the model.
Statistics focuses on making inferences (including estimation and hypothesis testing)
about a model based on observed data, using probability theory.
1.1 Random experiments
In probability theory, an experiment is any procedure that can be infinitely re-
peated and has a well-defined set of possible outcomes.
An experiment is said to be random if it has more than one possible outcome,
and deterministic if it has only one.
• Flip a coin
• Roll a die
There are two common features for random experiments:
• The outcome cannot be predicted with certainty.
• All the possible outcomes are known with certainty.
1.2 Sample space
For a random experiment, an observed result of interest is called an outcome.
We refer to the set of all possible distinct outcomes of a random experiment as the
sample space (usually denoted as S).
Groupsorsetsofoutcomes(i.e.,subsetsofthesamples

In [119]:
!pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 3.9 MB/s eta 0:00:00a 0:00:01


In [121]:
import docx

def extract_text_from_docx(docx_path):
    doc = docx.Document(docx_path)
    text = []
    for para in doc.paragraphs:
        text.append(para.text)
    
    return "\n".join(text)

docx_path = "/Users/arpitsrivastav/Desktop/hack-the-north-2024/Stat240Fall2024W1.docx"
docx_text = extract_text_from_docx(docx_path)
print(docx_text)


STAT 240 Week 1 Summary

Probability (vs Statistics)
Probability starts with a model, and it studies properties of the model.
Statistics focuses on making inferences (including estimation and hypothesis testing) about a model based on observed data, using probability theory.
Random experiments
In probability theory, an experiment is any procedure that can be infinitely re- peated and has a well-defined set of possible outcomes.
An experiment is said to be random if it has more than one possible outcome, and deterministic if it has only one.
Flip a coin
Roll a die
There are two common features for random experiments:
The outcome cannot be predicted with certainty.
All the possible outcomes are known with certainty.
Sample space
For a random experiment, an observed result of interest is called an outcome.
We refer to the set of all possible distinct outcomes of a random experiment as the
sample space (usually denoted as S).
Groups or sets of outcomes (i.e., subsets of the sample space) w

In [123]:
!pip install google-cloud-speech


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.4/292.4 kB 5.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.9/200.9 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.9/220.9 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 414.7/414.7 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 8.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.64.1
    Uninstalling grpcio-1.64.1:
      Successfully uninstalled grpcio-1.64.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installe

In [137]:
!pip install assemblyai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.7/72.7 kB 715.4 kB/s eta 0:00:00--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.8/148.8 kB 366.3 kB/s eta 0:00:00a 0:00:01


In [141]:
import assemblyai as aai
api_key = "e055223b479b40d59f513b946e994a48" 
def transcribe_audio(api_key, audio_file):
    aai.settings.api_key = api_key

    transcriber = aai.Transcriber()
    transcript = transcriber.transcribe(audio_file)
    if transcript.error:
        print(transcript.error)
        return
    print(transcript.text)


In [143]:
!pip install transformers

In [145]:
!pip install python-pptx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 2.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 7.1 MB/s eta 0:00:00


In [147]:
from pptx import Presentation

def pptx_to_text(pptx_file):
    prs = Presentation(pptx_file)
    
    all_text = []
    
    for slide in prs.slides:
        for shape in slide.shapes:
            if hasattr(shape, "text"):
                all_text.append(shape.text)
    return "\n".join(all_text)

pptx_file = "test.pptx"
text_content = pptx_to_text(pptx_file)
print(text_content)


AI Financial Saving 
Coach App
By group 505: Yixin Cai, Yujin Bae, Raghav Joshi, Arpit Srivastav, Rodney Dong


Venus
Hook here
30%
70%
Mercury
It’s the closest planet to the Sun and the smallest in the Solar System
Despite being red, Mars is actually a cold place. It’s full of iron oxide dust
Follow the link in the graph to modify its data and then paste the new one here. For more info, click here




Data, Research, and Pipeline
01
Our Solution
Business Case
Ethical Consideration & Future Potential
04
02
05
TABLE OF CONTENTS


Impact and Risk
Q&A
03
06

Saving Coach AI App
Our Solution
01
“How can AI help the average person save more and improve their long-term well-being?”
Financial stress
Lack of Financial Literacy
Behavioral barriers
Ineffective Budgeting Tools
Saving is hard!
Chatty Saver
AI Chatbot Saving Coach App
Azure AI Bot Services + OpenAI Services (GPT3.5)
Easily approachable conversational style
Chat and notification outputs for accessibility
Chatbot!
Personalized!
Pers

In [151]:
!pip install sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 1.3 MB/s eta 0:00:0000:0100:01


In [153]:
import re

from transformers import DonutProcessor, VisionEncoderDecoderModel
from datasets import load_dataset
import torch

processor = DonutProcessor.from_pretrained("naver-clova-ix/donut-base-finetuned-rvlcdip")
model = VisionEncoderDecoderModel.from_pretrained("naver-clova-ix/donut-base-finetuned-rvlcdip")

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
# load document image
dataset = load_dataset("hf-internal-testing/example-documents", split="test")
image = dataset[1]["image"]

# prepare decoder inputs
task_prompt = "<s_rvlcdip>"
decoder_input_ids = processor.tokenizer(task_prompt, add_special_tokens=False, return_tensors="pt").input_ids

pixel_values = processor(image, return_tensors="pt").pixel_values

outputs = model.generate(
    pixel_values.to(device),
    decoder_input_ids=decoder_input_ids.to(device),
    max_length=model.decoder.config.max_position_embeddings,
    pad_token_id=processor.tokenizer.pad_token_id,
    eos_token_id=processor.tokenizer.eos_token_id,
    use_cache=True,
    bad_words_ids=[[processor.tokenizer.unk_token_id]],
    return_dict_in_generate=True,
)

sequence = processor.batch_decode(outputs.sequences)[0]
sequence = sequence.replace(processor.tokenizer.eos_token, "").replace(processor.tokenizer.pad_token, "")
sequence = re.sub(r"<.*?>", "", sequence, count=1).strip()  # remove first task start token
print(processor.token2json(sequence))

ValueError: Cannot instantiate this tokenizer from a slow version. If it's based on sentencepiece, make sure you have sentencepiece installed.